In [4]:
import pandas as pd
import json

with open(r"D:\AI Projects\powerthon\powerthon_AI\data\outputs\record_0005.json") as f:
    data = json.load(f)

df = pd.json_normalize(data["properties"])
print(df)


  readingId.type readingId.description        readingId.value timestamp.type  \
0         string  Idempotent unique id  read-0005-I2128437297         string   

  timestamp.format       timestamp.value timezone.type  \
0        date-time  2025-10-15T04:09:11Z        string   

        timezone.description timezone.value source.type  ...  \
0  IANA tz e.g. Asia/Kolkata            UTC      object  ...   

  battery.properties.cycles.type battery.properties.cycles.value  \
0                           null                            None   

  battery.required extensions.type extensions.additionalProperties.type  \
0        [present]          object                               object   

  ingestedAt.type ingestedAt.format      ingestedAt.value signature.type  \
0          string         date-time  2025-10-17T05:15:11Z           null   

  signature.value  
0            None  

[1 rows x 168 columns]


2.validate json based on schema 

In [7]:
# import json
# from jsonschema import validate, ValidationError

# # Load schema
# with open(r"D:\AI Projects\powerthon\powerthon_AI\app\config\prompt.yaml") as f:
#     schema = yaml.load(f)

# # Load JSON data
# with open(r"D:\AI Projects\powerthon\powerthon_AI\data\outputs\record_0005.json") as f:
#     data = json.load(f)

# # Validate each record
# for record in data:
#     try:
#         validate(instance=record, schema=schema)
#     except ValidationError as e:
#         print("Invalid record:", record)
#         print("Error:", e.message)


import json
import yaml
from jsonschema import validate, ValidationError

# Load schema from YAML file
with open(r"D:\AI Projects\powerthon\powerthon_AI\app\config\solarcanonical.yaml", "r") as f:
    schema = yaml.safe_load(f)

# Load JSON data
with open(r"D:\AI Projects\powerthon\powerthon_AI\data\outputs\record_0005.json", "r") as f:
    data = json.load(f)

# Validate each record against schema
for record in data:
    try:
        validate(instance=record, schema=schema)
        print(" Valid record:", record)
    except ValidationError as e:
        print(" Invalid record:", record)
        print("Error:", e.message)



 Invalid record: title
Error: 'title' is not of type 'object'
 Invalid record: schemaVersion
Error: 'schemaVersion' is not of type 'object'
 Invalid record: type
Error: 'type' is not of type 'object'
 Invalid record: properties
Error: 'properties' is not of type 'object'
 Invalid record: required
Error: 'required' is not of type 'object'


In [8]:
import json
import yaml
from jsonschema import validate, ValidationError

# --- Load schema ---
with open(r"D:\AI Projects\powerthon\powerthon_AI\app\config\solarcanonical.yaml", "r") as f:
    schema = yaml.safe_load(f)

# --- Load JSON data ---
with open(r"D:\AI Projects\powerthon\powerthon_AI\data\outputs\record_0005.json", "r") as f:
    data = json.load(f)

# --- Recursive function to extract 'value' ---
def extract_values(node):
    if isinstance(node, dict):
        # If there's a 'value' key, return it directly
        if "value" in node:
            return extract_values(node["value"])
        # Otherwise, recursively go through sub-properties
        else:
            return {k: extract_values(v) for k, v in node.items() if k not in ("type", "format", "description", "required", "properties", "items")}
    elif isinstance(node, list):
        return [extract_values(item) for item in node]
    else:
        return node

# --- Flatten input JSON to get pure data ---
flat_data = extract_values(data["properties"])

# --- Validate ---
try:
    validate(instance=flat_data, schema=schema)
    print(" JSON is valid according to the schema.")
except ValidationError as e:
    print(" Validation failed:")
    print("Path:", list(e.path))
    print("Error:", e.message)


 Validation failed:
Path: ['power']
Error: 'acActive' is a required property
